In [3]:
from dotenv import load_dotenv
from dspy.teleprompt import GEPA
import os
load_dotenv()

import dspy
from aime_dataset import init_dataset
from multi_llm_proposer import MultiLLMProposalFn

train_set, val_set, test_set = init_dataset()

# Configure base LM
lm = dspy.LM("openai/gpt-4.1-mini", temperature=1, max_tokens=32000)
dspy.configure(lm=lm)
os.environ["DSPY_CACHE_DIR"] = "/tmp/dspy_cache"
os.makedirs("/tmp/dspy_cache", exist_ok=True)



In [5]:
# Define the program (same as gepa_test.ipynb)
class GenerateResponse(dspy.Signature):
    """Solve the problem and provide the answer in the correct format."""
    problem = dspy.InputField()
    answer = dspy.OutputField()

program = dspy.ChainOfThought(GenerateResponse)

# Define metric with feedback (same as gepa_test.ipynb)
def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    correct_answer = int(example['answer'])
    written_solution = example.get('solution', '')
    try:
        llm_answer = int(prediction.answer)
    except ValueError as e:
        feedback_text = f"The final answer must be a valid integer and nothing else. You responded with '{prediction.answer}', which couldn't be parsed as a python integer. Please ensure your answer is a valid integer without any additional text or formatting."
        feedback_text += f" The correct answer is '{correct_answer}'."
        if written_solution:
            feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems and ensure your final answer is a valid integer."
        return dspy.Prediction(score=0, feedback=feedback_text)

    score = int(correct_answer == llm_answer)

    feedback_text = ""
    if score == 1:
        feedback_text = f"Your answer is correct. The correct answer is '{correct_answer}'."
    else:
        feedback_text = f"Your answer is incorrect. The correct answer is '{correct_answer}'."
    
    if written_solution:
        feedback_text += f" Here's the full step-by-step solution:\n{written_solution}\n\nThink about what takeaways you can learn from this solution to improve your future answers and approach to similar problems."

    return dspy.Prediction(score=score, feedback=feedback_text)

proposer = MultiLLMProposalFn(
    proposal_lms=[
        dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),  # Reasoning model proposal
        dspy.LM("openrouter/anthropic/claude-sonnet-4.5", temperature=1.0, max_tokens=16000), 
        dspy.LM("openrouter/google/gemini-2.5-flash", temperature=0.6, max_tokens=16000),
    ],
    judge_lm=dspy.LM("openrouter/anthropic/claude-sonnet-4.5", temperature=1.0, max_tokens=16000), 
    merger_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000), 
    top_n=2,  
    verbose=True,
)

In [ ]:

optimizer = GEPA(
    metric=metric_with_feedback,
    reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),  
    max_full_evals=1,
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=5,
    instruction_proposer=proposer,
)

print("Starting GEPA optimization with MultiLLMProposalFn...")
print(f"Training set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")

optimized_program = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)

print(optimized_program.predict.signature.instructions)

In [ ]:
# Evaluate the optimized program on test set
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
    display_table=True,
    display_progress=True
)

print("\nEvaluating optimized program...")
result = evaluate(optimized_program)
print(f"\nFinal score: {result.score}%")

In [ ]:
# Run GEPA optimization with DEFAULT proposer (single LLM)
optimizer_default = GEPA(
    metric=metric_with_feedback,
    reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),  # Single LLM for proposals
    max_full_evals=1,
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=3,
)

optimized_program_default = optimizer_default.compile(
    program,
    trainset=train_set,
    valset=val_set,
)


In [ ]:
# Evaluate the optimized program on test set
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
    display_table=True,
    display_progress=True
)

print("\nEvaluating optimized program...")
result = evaluate(optimized_program_default)
print(f"\nFinal score: {result.score}%")

In [ ]:

# Initialize proposer
proposer = MultiLLMProposalFn(
    proposal_lms=[
        dspy.LM("openai/gpt-5", temperature=1, max_tokens=16000),
        dspy.LM("openrouter/anthropic/claude-sonnet-4.5", temperature=0.75, max_tokens=16000),
    ],
    judge_lm=dspy.LM("openai/gpt-5", temperature=1, max_tokens=16000),
    merger_lm=dspy.LM("openai/gpt-5", temperature=1, max_tokens=16000),
    top_n=2,
    verbose=True
)

optimizer = dspy.GEPA(
    metric=metric_with_feedback,
    reflection_lm=dspy.LM("openai/gpt-5", temperature=1, max_tokens=16000),  
    auto="light",
    instruction_proposer=proposer,
    num_threads=32,
    track_stats=True,
    reflection_minibatch_size=3,
)

optimized_program_multi = optimizer.compile(
    program,
    trainset=train_set,
    valset=val_set,
)


In [ ]:
# Evaluate the optimized program on test set
evaluate = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
    display_table=True,
    display_progress=True
)

print("\nEvaluating optimized program...")
result = evaluate(optimized_program_multi)
print(f"\nFinal score: {result.score}%")

In [6]:
# Run MultiLLMProposalFn version 5 times and collect results
import time
from datetime import datetime

print("="*60)
print("RUNNING MULTI-LLM VERSION 5 TIMES")
print("="*60)

multi_llm_results = []
evaluate_fn = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
)

for run_num in range(1, 6):
    print(f"\n{'='*60}")
    print(f"RUN {run_num}/5 - MultiLLMProposalFn")
    print(f"{'='*60}")
    start_time = time.time()
    
    # Create optimizer for this run
    optimizer = GEPA(
        metric=metric_with_feedback,
        reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),  
        max_full_evals=1,
        num_threads=32,
        track_stats=True,
        reflection_minibatch_size=5,
        instruction_proposer=proposer,
    )
    
    # Optimize
    optimized_program = optimizer.compile(
        program,
        trainset=train_set,
        valset=val_set,
    )
    
    # Evaluate
    result = evaluate_fn(optimized_program)
    elapsed_time = time.time() - start_time
    
    multi_llm_results.append({
        'run': run_num,
        'score': result.score,
        'correct': result.score * len(test_set) / 100,
        'total': len(test_set),
        'time': elapsed_time
    })
    
    print(f"Run {run_num} complete: {result.score:.2f}% ({result.score * len(test_set) / 100:.0f}/{len(test_set)}) in {elapsed_time/60:.1f} min")

# Calculate averages
multi_avg_score = sum(r['score'] for r in multi_llm_results) / len(multi_llm_results)
multi_avg_correct = sum(r['correct'] for r in multi_llm_results) / len(multi_llm_results)
multi_avg_time = sum(r['time'] for r in multi_llm_results) / len(multi_llm_results)
multi_std_score = (sum((r['score'] - multi_avg_score)**2 for r in multi_llm_results) / len(multi_llm_results))**0.5

print(f"\n{'='*60}")
print("MULTI-LLM RESULTS SUMMARY")
print(f"{'='*60}")
print(f"Average Score: {multi_avg_score:.2f}% ± {multi_std_score:.2f}%")
print(f"Average Correct: {multi_avg_correct:.1f}/{len(test_set)}")
print(f"Average Time: {multi_avg_time/60:.1f} minutes")
print(f"\nIndividual runs:")
for r in multi_llm_results:
    print(f"  Run {r['run']}: {r['score']:.2f}% ({r['correct']:.0f}/{r['total']})")


2025/11/05 22:54:09 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 90 metric calls of the program. This amounts to 1.00 full evals on the train+val set.
2025/11/05 22:54:09 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.


RUNNING MULTI-LLM VERSION 5 TIMES

RUN 1/5 - MultiLLMProposalFn


GEPA Optimization:   0%|          | 0/90 [00:00<?, ?rollouts/s]2025/11/05 22:54:09 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/05 22:54:09 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
GEPA Optimization:  50%|█████     | 45/90 [00:00<00:00, 102.52rollouts/s]2025/11/05 22:54:09 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 4.00 / 5 (80.0%): 100%|██████████| 5/5 [00:00<00:00, 575.11it/s]

2025/11/05 22:54:09 INFO dspy.evaluate.evaluate: Average Metric: 4.0 / 5 (80.0%)
2025/11/05 22:54:09 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: Input
- problem: a single math problem statement (often contest-style).

Your task
- Solve the problem correctly and present the solution in this strict two-part output:
  1) A concise reasoning section.
  2) A final answer section containing only the final result (no words, symbols, or extra text).

Output format
- Exactly:
  ### reasoning
  <concise, step-by-step reasoning with exact arithmetic and clear logic; show only essential steps and checks>
  ### answer
  <final result only, e.g., 227>

Preflight (before solving)
- Parse what the problem actually asks for (e.g., remainder mod m, last 3 digits, integer part). Apply any required final transformation to the computed result.
- Clarify whether solutions are ordered or unordered; unless stated otherwise, treat tuples (a,b,c,…) as ordered.
- Note all constrai

...



Processing component: predict

Generating 3 proposals in parallel...
  [Proposal 1] Generated with openai/gpt-5
  [Proposal 3] Generated with openrouter/google/gemini-2.5-flash
  [Proposal 2] Generated with openrouter/anthropic/claude-sonnet-4.5

Scoring 3 proposals with judge LLM...
  [Proposal 1] Score: 91.5/100 (Dataset: 47.5, Quality: 44.0)
  [Proposal 2] Score: 72.0/100 (Dataset: 34.0, Quality: 38.0)
  [Proposal 3] Score: 60.0/100 (Dataset: 28.0, Quality: 32.0)

Selected top 2 proposals for merging:
  1. Score: 91.5/100
  2. Score: 72.0/100

Merging top 2 proposals...
  Merged instruction created (5465 chars)
  Rationale: 1) Unique elements taken from each proposal and why
- From Proposal 1:
  - Strict two-part output format with “### reasoning” and “### answer,” and the requirement that the answer line contain only th...

[Final] New instruction for predict:
  Input
- problem: a single math problem statement (often contest-style).

Your task
- Solve the problem correctly and pr

2025/11/05 22:58:20 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 5 (40.0%)
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Proposed new text for predict: You are solving AIME/AMC-style contest math problems.

Output format (required):
- reasoning: Concise but complete derivation with the decisive steps, formulas, and case structure. No extraneous commentary.
- answer: Only the final value requested (integer, simplified radical/fraction, or derived quantity like m+n), no extra words.

Before solving: identify the problem type and goal
- Is it maximize/minimize vs compute directly?
- Is it over a family (“smallest sphere containing each box” => pick the worst-case instance) or a single instance?
- Does the answer require a derived quantity (e.g., m+n from p/q)? If so, reduce p/q to lowest terms first.
- Are there universal items/always-present sets that change how “exactly k” is counted?
- In geometry, is there an implicit coordinate orientation or perpendicularity



Processing component: predict

Generating 3 proposals in parallel...
  [Proposal 1] Generated with openai/gpt-5
  [Proposal 2] Generated with openrouter/anthropic/claude-sonnet-4.5
  [Proposal 3] Generated with openrouter/google/gemini-2.5-flash

Scoring 3 proposals with judge LLM...
  [Proposal 1] Score: 70.0/100 (Dataset: 38.0, Quality: 32.0)
  [Proposal 2] Score: 77.0/100 (Dataset: 38.0, Quality: 39.0)
  [Proposal 3] Score: 52.0/100 (Dataset: 22.0, Quality: 30.0)

Selected top 2 proposals for merging:
  1. Score: 77.0/100
  2. Score: 70.0/100

Merging top 2 proposals...
  Merged instruction created (6877 chars)
  Rationale: 1) Unique elements used from each proposal and why
- From Proposal 1:
  • Clear structure by problem type and essential formulas (space diagonal and r = d/2; box SA and volume identities; Vieta). Thes...

[Final] New instruction for predict:
  You are solving AIME/AMC-style contest math problems.

Output format (required):
- reasoning: Concise but complete deri

2025/11/05 22:58:20 INFO dspy.evaluate.evaluate: Average Metric: 18.0 / 45 (40.0%)
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Full valset score for new program: 0.4
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Full train_val score for new program: 0.4
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Individual valset scores for new program: [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: New valset pareto front scores: [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Full valset pareto front score: 0.5555555555555556
2025/11/05 22:58:20 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Updated valset pareto front programs: [{0, 1}, {0

Run 5 complete: 30.00% (45/150) in 0.0 min

MULTI-LLM RESULTS SUMMARY
Average Score: 32.27% ± 4.53%
Average Correct: 48.4/150
Average Time: 0.8 minutes

Individual runs:
  Run 1: 41.33% (62/150)
  Run 2: 30.00% (45/150)
  Run 3: 30.00% (45/150)
  Run 4: 30.00% (45/150)
  Run 5: 30.00% (45/150)


In [7]:
# Run DEFAULT GEPA version 5 times and collect results

print("="*60)
print("RUNNING DEFAULT GEPA VERSION 5 TIMES")
print("="*60)

default_results = []
evaluate_fn_default = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
    display_table=False,  # Don't show table for each run
    display_progress=False,  # Don't show progress for each run
)

for run_num in range(1, 6):
    print(f"\n{'='*60}")
    print(f"RUN {run_num}/5 - Default GEPA")
    print(f"{'='*60}")
    start_time = time.time()
    
    # Create optimizer for this run (default, no instruction_proposer)
    optimizer_default = GEPA(
        metric=metric_with_feedback,
        reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),
        max_full_evals=1,
        num_threads=32,
        track_stats=True,
        reflection_minibatch_size=3,
    )
    
    # Optimize
    optimized_program_default = optimizer_default.compile(
        program,
        trainset=train_set,
        valset=val_set,
    )
    
    # Evaluate
    result = evaluate_fn_default(optimized_program_default)
    elapsed_time = time.time() - start_time
    
    default_results.append({
        'run': run_num,
        'score': result.score,
        'correct': result.score * len(test_set) / 100,
        'total': len(test_set),
        'time': elapsed_time
    })
    
    print(f"Run {run_num} complete: {result.score:.2f}% ({result.score * len(test_set) / 100:.0f}/{len(test_set)}) in {elapsed_time/60:.1f} min")

# Calculate averages
default_avg_score = sum(r['score'] for r in default_results) / len(default_results)
default_avg_correct = sum(r['correct'] for r in default_results) / len(default_results)
default_avg_time = sum(r['time'] for r in default_results) / len(default_results)
default_std_score = (sum((r['score'] - default_avg_score)**2 for r in default_results) / len(default_results))**0.5

print(f"\n{'='*60}")
print("DEFAULT GEPA RESULTS SUMMARY")
print(f"{'='*60}")
print(f"Average Score: {default_avg_score:.2f}% ± {default_std_score:.2f}%")
print(f"Average Correct: {default_avg_correct:.1f}/{len(test_set)}")
print(f"Average Time: {default_avg_time/60:.1f} minutes")
print(f"\nIndividual runs:")
for r in default_results:
    print(f"  Run {r['run']}: {r['score']:.2f}% ({r['correct']:.0f}/{r['total']})")


2025/11/05 23:05:13 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 90 metric calls of the program. This amounts to 1.00 full evals on the train+val set.
2025/11/05 23:05:13 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.


RUNNING DEFAULT GEPA VERSION 5 TIMES

RUN 1/5 - Default GEPA


GEPA Optimization:   0%|          | 0/90 [00:00<?, ?rollouts/s]2025/11/05 23:05:13 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/05 23:05:13 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
2025/11/05 23:05:13 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 3311.29it/s] 

2025/11/05 23:05:13 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/11/05 23:05:13 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You will be given an input block with a single field:
- problem: a self-contained math problem (often contest-style).

Your task:
1) Solve the problem correctly.
2) Output your result in the exact two-section format:
   - A concise solution under a "reasoning" section.
   - The final result alone under an "answer" section.

Output format (must match exactly):
### reasoning
<your concise derivation and checks>

### answer
<final result only, no extra words>

Guidelines for solving and presenting:

General
- Read carefully what is being asked (e.g., largest/smallest, count of ordered triples, uniqueness).
- Prefer clean algebraic/symmetric identities, modular arithmetic, and structural reasoning over brute force. If a small finite set remains, enumerate systematically.
- Verify all constraints (domains, digit rang

...

GEPA Optimization:   0%|          | 0/90 [00:00<?, ?rollouts/s]2025/11/05 23:11:30 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/05 23:11:30 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
2025/11/05 23:11:30 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 3421.13it/s] 

2025/11/05 23:11:30 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/11/05 23:11:30 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You will be given an input block with a single field:
- problem: a self-contained math problem (often contest-style).

Your task:
1) Solve the problem correctly.
2) Output your result in the exact two-section format:
   - A concise solution under a "reasoning" section.
   - The final result alone under an "answer" section.

Output format (must match exactly):
### reasoning
<your concise derivation and checks>

### answer
<final result only, no extra words>

Guidelines for solving and presenting:

General
- Read carefully what is being asked (e.g., largest/smallest, count of ordered triples, uniqueness).
- Prefer clean algebraic/symmetric identities, modular arithmetic, and structural reasoning over brute force. If a small finite set remains, enumerate systematically.
- Verify all constraints (domains, digit rang

2025/11/05 23:11:31 INFO dspy.evaluate.evaluate: Average Metric: 45 / 150 (30.0%)


Run 5 complete: 30.00% (45/150) in 0.0 min

DEFAULT GEPA RESULTS SUMMARY
Average Score: 30.00% ± 0.00%
Average Correct: 45.0/150
Average Time: 1.3 minutes

Individual runs:
  Run 1: 30.00% (45/150)
  Run 2: 30.00% (45/150)
  Run 3: 30.00% (45/150)
  Run 4: 30.00% (45/150)
  Run 5: 30.00% (45/150)


In [9]:
# Final comparison of both approaches
print("="*80)
print("FINAL COMPARISON: MultiLLMProposalFn vs Default GEPA (5 runs each)")
print("="*80)

print(f"\n{'Metric':<30} {'MultiLLM':<25} {'Default':<25} {'Difference':<15}")
print("-" * 95)
print(f"{'Average Score':<30} {multi_avg_score:>6.2f}% ± {multi_std_score:>5.2f}%{'':>10} {default_avg_score:>6.2f}% ± {default_std_score:>5.2f}%{'':>10} {multi_avg_score - default_avg_score:>+6.2f}%")
print(f"{'Average Correct':<30} {multi_avg_correct:>6.1f}/{len(test_set):<4}{'':>15} {default_avg_correct:>6.1f}/{len(test_set):<4}{'':>15} {multi_avg_correct - default_avg_correct:>+6.1f}")
print(f"{'Average Time':<30} {multi_avg_time/60:>6.1f} min{'':>15} {default_avg_time/60:>6.1f} min{'':>15} {(multi_avg_time - default_avg_time)/60:>+6.1f} min")

print(f"\n{'Individual Run Scores':<30} {'MultiLLM':<25} {'Default':<25}")
print("-" * 80)
for i in range(5):
    print(f"{'Run ' + str(i+1):<30} {multi_llm_results[i]['score']:>6.2f}%{'':>15} {default_results[i]['score']:>6.2f}%")

print(f"\n{'='*80}")
if multi_avg_score > default_avg_score:
    improvement = ((multi_avg_score / default_avg_score) - 1) * 100
    print(f"✓ MultiLLMProposalFn is BETTER by {multi_avg_score - default_avg_score:.2f} percentage points")
    print(f"  Relative improvement: {improvement:+.2f}%")
elif default_avg_score > multi_avg_score:
    improvement = ((default_avg_score / multi_avg_score) - 1) * 100
    print(f"✓ Default GEPA is BETTER by {default_avg_score - multi_avg_score:.2f} percentage points")
    print(f"  Relative improvement: {improvement:+.2f}%")
else:
    print("Both approaches perform equally well")
print(f"{'='*80}")


FINAL COMPARISON: MultiLLMProposalFn vs Default GEPA (5 runs each)

Metric                         MultiLLM                  Default                   Difference     
-----------------------------------------------------------------------------------------------
Average Score                   32.27% ±  4.53%            30.00% ±  0.00%            +2.27%
Average Correct                  48.4/150                   45.0/150                   +3.4
Average Time                      0.8 min                   1.3 min                  -0.4 min

Individual Run Scores          MultiLLM                  Default                  
--------------------------------------------------------------------------------
Run 1                           41.33%                 30.00%
Run 2                           30.00%                 30.00%
Run 3                           30.00%                 30.00%
Run 4                           30.00%                 30.00%
Run 5                           30.00%       

In [10]:
# Run MultiLLMProposalFn version 5 times and collect results
import time
from datetime import datetime

print("="*60)
print("RUNNING MULTI-LLM VERSION 5 TIMES")
print("="*60)

multi_llm_results = []
evaluate_fn = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
)

for run_num in range(1, 6):
    print(f"\n{'='*60}")
    print(f"RUN {run_num}/5 - MultiLLMProposalFn")
    print(f"{'='*60}")
    start_time = time.time()
    
    # Create optimizer for this run
    optimizer = GEPA(
        metric=metric_with_feedback,
        reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000), 
        auto="light",
        num_threads=32,
        track_stats=True,
        reflection_minibatch_size=5,
        instruction_proposer=proposer,
    )
    
    # Optimize
    optimized_program = optimizer.compile(
        program,
        trainset=train_set,
        valset=val_set,
    )
    
    # Evaluate
    result = evaluate_fn(optimized_program)
    elapsed_time = time.time() - start_time
    
    multi_llm_results.append({
        'run': run_num,
        'score': result.score,
        'correct': result.score * len(test_set) / 100,
        'total': len(test_set),
        'time': elapsed_time
    })
    
    print(f"Run {run_num} complete: {result.score:.2f}% ({result.score * len(test_set) / 100:.0f}/{len(test_set)}) in {elapsed_time/60:.1f} min")

# Calculate averages
multi_avg_score = sum(r['score'] for r in multi_llm_results) / len(multi_llm_results)
multi_avg_correct = sum(r['correct'] for r in multi_llm_results) / len(multi_llm_results)
multi_avg_time = sum(r['time'] for r in multi_llm_results) / len(multi_llm_results)
multi_std_score = (sum((r['score'] - multi_avg_score)**2 for r in multi_llm_results) / len(multi_llm_results))**0.5

print(f"\n{'='*60}")
print("MULTI-LLM RESULTS SUMMARY")
print(f"{'='*60}")
print(f"Average Score: {multi_avg_score:.2f}% ± {multi_std_score:.2f}%")
print(f"Average Correct: {multi_avg_correct:.1f}/{len(test_set)}")
print(f"Average Time: {multi_avg_time/60:.1f} minutes")
print(f"\nIndividual runs:")
for r in multi_llm_results:
    print(f"  Run {r['run']}: {r['score']:.2f}% ({r['correct']:.0f}/{r['total']})")


2025/11/05 23:16:10 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 560 metric calls of the program. This amounts to 6.22 full evals on the train+val set.
2025/11/05 23:16:10 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.


RUNNING MULTI-LLM VERSION 5 TIMES

RUN 1/5 - MultiLLMProposalFn


GEPA Optimization:   0%|          | 0/560 [00:00<?, ?rollouts/s]2025/11/05 23:16:10 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/05 23:16:10 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
2025/11/05 23:16:10 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 4.00 / 5 (80.0%): 100%|██████████| 5/5 [00:00<00:00, 3494.09it/s] 

2025/11/05 23:16:10 INFO dspy.evaluate.evaluate: Average Metric: 4.0 / 5 (80.0%)
2025/11/05 23:16:10 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: Input
- problem: a single math problem statement (often contest-style).

Your task
- Solve the problem correctly and present the solution in this strict two-part output:
  1) A concise reasoning section.
  2) A final answer section containing only the final result (no words, symbols, or extra text).

Output format
- Exactly:
  ### reasoning
  <concise, step-by-step reasoning with exact arithmetic and clear logic; show only essential steps and checks>
  ### answer
  <final result only, e.g., 227>

Preflight (before solving)
- Parse what the problem actually asks for (e.g., remainder mod m, last 3 digits, integer part). Apply any required final transformation to the computed result.
- Clarify whether solutions are ordered or unordered; unless stated otherwise, treat tuples (a,b,c,…) as ordered.
- Note all constrai

...



Processing component: predict

Generating 3 proposals in parallel...
  [Proposal 1] Generated with openai/gpt-5
  [Proposal 2] Generated with openrouter/anthropic/claude-sonnet-4.5
  [Proposal 3] Generated with openrouter/google/gemini-2.5-flash

Scoring 3 proposals with judge LLM...
  [Proposal 1] Score: 82.0/100 (Dataset: 42.0, Quality: 40.0)
  [Proposal 2] Score: 77.0/100 (Dataset: 38.0, Quality: 39.0)
  [Proposal 3] Score: 55.0/100 (Dataset: 25.0, Quality: 30.0)

Selected top 2 proposals for merging:
  1. Score: 82.0/100
  2. Score: 77.0/100

Merging top 2 proposals...


2025/11/06 01:32:43 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Proposed new text for predict: AIME/AMC contest-math solver — strict output, answer-type gate, and complete method

Output format (required)
- reasoning: Concise but complete derivation with decisive steps, exact symbolic manipulations, and clean case structure. No fluff. Avoid decimals unless forced; keep radicals/fractions exact and reduced.
- answer: Only the final requested value. Digits only if an integer. No words, LaTeX, zero-padding, or punctuation.

Answer-type gate (apply before and after solving)
- Default: AIME answers are integers in [0, 999]. If the prompt requests a derived integer (e.g., m+n from reduced p/q or from a radical), reduce/simplify first, then output that integer.
- If the prompt explicitly requests a non-integer form (rare in AMC/AIME-style tasks), output the exact simplified value; otherwise, an AIME-style result must be an integer. If you obtain a non-integer where an AIME integer is expect

  Merged instruction created (6602 chars)
  Rationale: 1) Unique elements taken and why
- From Proposal 1:
  • Strict answer-line rules (digits only; no LaTeX/words), and explicit AIME sanity (0–999). This directly fixes format parsing failures in Example...

[Final] New instruction for predict:
  AIME/AMC contest-math solver — strict output, answer-type gate, and complete method

Output format (required)
- reasoning: Concise but complete derivation with decisive steps, exact symbolic manipulat...


2025/11/06 01:32:43 INFO dspy.evaluate.evaluate: Average Metric: 65 / 150 (43.3%)


Run 5 complete: 43.33% (65/150) in 0.0 min

MULTI-LLM RESULTS SUMMARY
Average Score: 44.40% ± 2.48%
Average Correct: 66.6/150
Average Time: 27.3 minutes

Individual runs:
  Run 1: 49.33% (74/150)
  Run 2: 42.67% (64/150)
  Run 3: 43.33% (65/150)
  Run 4: 43.33% (65/150)
  Run 5: 43.33% (65/150)


In [ ]:
print("="*60)
print("RUNNING DEFAULT GEPA VERSION 5 TIMES")
print("="*60)

default_results = []
evaluate_fn_default = dspy.Evaluate(
    devset=test_set,
    metric=lambda example, prediction, trace=None, pred_name=None, pred_trace=None: 
        int(int(example['answer']) == int(prediction.answer)) if str(prediction.answer).strip().isdigit() else 0,
    num_threads=32,
    display_table=False, 
    display_progress=False, 
)

for run_num in range(1, 6):
    print(f"\n{'='*60}")
    print(f"RUN {run_num}/5 - Default GEPA")
    print(f"{'='*60}")
    start_time = time.time()
    
    optimizer_default = GEPA(
        metric=metric_with_feedback,
        auto="light",
        reflection_lm=dspy.LM("openai/gpt-5", temperature=1.0, max_tokens=16000),
        num_threads=32,
        track_stats=True,
        reflection_minibatch_size=3,
    )
    
    optimized_program_default = optimizer_default.compile(
        program,
        trainset=train_set,
        valset=val_set,
    )
    
    result = evaluate_fn_default(optimized_program_default)
    elapsed_time = time.time() - start_time
    
    default_results.append({
        'run': run_num,
        'score': result.score,
        'correct': result.score * len(test_set) / 100,
        'total': len(test_set),
        'time': elapsed_time
    })
    
    print(f"Run {run_num} complete: {result.score:.2f}% ({result.score * len(test_set) / 100:.0f}/{len(test_set)}) in {elapsed_time/60:.1f} min")

# Calculate averages
default_avg_score = sum(r['score'] for r in default_results) / len(default_results)
default_avg_correct = sum(r['correct'] for r in default_results) / len(default_results)
default_avg_time = sum(r['time'] for r in default_results) / len(default_results)
default_std_score = (sum((r['score'] - default_avg_score)**2 for r in default_results) / len(default_results))**0.5

print(f"\n{'='*60}")
print("DEFAULT GEPA RESULTS SUMMARY")
print(f"{'='*60}")
print(f"Average Score: {default_avg_score:.2f}% ± {default_std_score:.2f}%")
print(f"Average Correct: {default_avg_correct:.1f}/{len(test_set)}")
print(f"Average Time: {default_avg_time/60:.1f} minutes")
print(f"\nIndividual runs:")
for r in default_results:
    print(f"  Run {r['run']}: {r['score']:.2f}% ({r['correct']:.0f}/{r['total']})")


2025/11/06 01:32:43 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 560 metric calls of the program. This amounts to 6.22 full evals on the train+val set.


RUNNING DEFAULT GEPA VERSION 5 TIMES

RUN 1/5 - Default GEPA


2025/11/06 01:32:43 INFO dspy.teleprompt.gepa.gepa: Using 45 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.
GEPA Optimization:   0%|          | 0/560 [00:00<?, ?rollouts/s]2025/11/06 01:32:44 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/06 01:32:44 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.4444444444444444
2025/11/06 01:32:44 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.4444444444444444


Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 4070.82it/s]

2025/11/06 01:32:44 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/11/06 01:32:44 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: You will be given an input block with a single field:
- problem: a self-contained math problem (often contest-style).

Your task:
1) Solve the problem correctly.
2) Output your result in the exact two-section format:
   - A concise solution under a "reasoning" section.
   - The final result alone under an "answer" section.

Output format (must match exactly):
### reasoning
<your concise derivation and checks>

### answer
<final result only, no extra words>

Guidelines for solving and presenting:

General
- Read carefully what is being asked (e.g., largest/smallest, count of ordered triples, uniqueness).
- Prefer clean algebraic/symmetric identities, modular arithmetic, and structural reasoning over brute force. If a small finite set remains, enumerate systematically.
- Verify all constraints (domains, digit rang

...

2025/11/06 04:38:37 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 11: New subsample score 3 is better than old score 2. Continue to full eval and add to candidate pool.
2025/11/06 04:38:37 INFO dspy.evaluate.evaluate: Average Metric: 16.0 / 45 (35.6%)
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 11: Full valset score for new program: 0.35555555555555557
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 11: Full train_val score for new program: 0.35555555555555557
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 11: Individual valset scores for new program: [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 11: New valset pareto front scores: [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:00<00:00, 3746.03it/s]

2025/11/06 04:38:37 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Proposed new text for predict: Task
- You will be given a single math problem under the key "problem".
- Solve it and produce two top-level fields in your output:
  - reasoning: a concise, exact, step-by-step solution using sound methods (no unnecessary verbosity).
  - answer: ONLY the final result in its simplest exact form (no words, units, or extra punctuation).

Output format
- Provide exactly two sections labeled:
  - reasoning
  - answer
- The "answer" must be a single value or expression exactly as requested by the problem (e.g., an integer or a simplified expression). Do not include explanatory text.
- Do not pad integers with leading zeros unless the problem explicitly requests it.
- Ensure the "answer" is of the expected type (e.g., a bare integer if the problem asks for an integer; reduce fractions to lowest terms before computing

2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: New subsample score 3 is better than old score 2. Continue to full eval and add to candidate pool.
2025/11/06 04:38:37 INFO dspy.evaluate.evaluate: Average Metric: 20.0 / 45 (44.4%)
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Full valset score for new program: 0.4444444444444444
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Full train_val score for new program: 0.4444444444444444
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: Individual valset scores for new program: [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
2025/11/06 04:38:37 INFO dspy.teleprompt.gepa.gepa: Iteration 12: New valset pareto front scores: [0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0]
2025/11/06 04:38:37 INFO dspy.t

Run 5 complete: 40.00% (60/150) in 0.0 min

DEFAULT GEPA RESULTS SUMMARY
Average Score: 44.00% ± 5.76%
Average Correct: 66.0/150
Average Time: 37.2 minutes

Individual runs:
  Run 1: 42.67% (64/150)
  Run 2: 55.33% (83/150)
  Run 3: 42.00% (63/150)
  Run 4: 40.00% (60/150)
  Run 5: 40.00% (60/150)


In [ ]:
print("="*80)
print("FINAL COMPARISON: MultiLLMProposalFn vs Default GEPA (5 runs each)")
print("="*80)

print(f"\n{'Metric':<30} {'MultiLLM':<25} {'Default':<25} {'Difference':<15}")
print("-" * 95)
print(f"{'Average Score':<30} {multi_avg_score:>6.2f}% ± {multi_std_score:>5.2f}%{'':>10} {default_avg_score:>6.2f}% ± {default_std_score:>5.2f}%{'':>10} {multi_avg_score - default_avg_score:>+6.2f}%")
print(f"{'Average Correct':<30} {multi_avg_correct:>6.1f}/{len(test_set):<4}{'':>15} {default_avg_correct:>6.1f}/{len(test_set):<4}{'':>15} {multi_avg_correct - default_avg_correct:>+6.1f}")
print(f"{'Average Time':<30} {multi_avg_time/60:>6.1f} min{'':>15} {default_avg_time/60:>6.1f} min{'':>15} {(multi_avg_time - default_avg_time)/60:>+6.1f} min")

print(f"\n{'Individual Run Scores':<30} {'MultiLLM':<25} {'Default':<25}")
print("-" * 80)
for i in range(5):
    print(f"{'Run ' + str(i+1):<30} {multi_llm_results[i]['score']:>6.2f}%{'':>15} {default_results[i]['score']:>6.2f}%")

print(f"\n{'='*80}")
if multi_avg_score > default_avg_score:
    improvement = ((multi_avg_score / default_avg_score) - 1) * 100
    print(f"✓ MultiLLMProposalFn is BETTER by {multi_avg_score - default_avg_score:.2f} percentage points")
    print(f"  Relative improvement: {improvement:+.2f}%")
elif default_avg_score > multi_avg_score:
    improvement = ((default_avg_score / multi_avg_score) - 1) * 100
    print(f"✓ Default GEPA is BETTER by {default_avg_score - multi_avg_score:.2f} percentage points")
    print(f"  Relative improvement: {improvement:+.2f}%")
else:
    print("Both approaches perform equally well")
print(f"{'='*80}")


FINAL COMPARISON: MultiLLMProposalFn vs Default GEPA (5 runs each)

Metric                         MultiLLM                  Default                   Difference     
-----------------------------------------------------------------------------------------------
Average Score                   44.40% ±  2.48%            44.00% ±  5.76%            +0.40%
Average Correct                  66.6/150                   66.0/150                   +0.6
Average Time                     27.3 min                  37.2 min                  -9.9 min

Individual Run Scores          MultiLLM                  Default                  
--------------------------------------------------------------------------------
Run 1                           49.33%                 42.67%
Run 2                           42.67%                 55.33%
Run 3                           43.33%                 42.00%
Run 4                           43.33%                 40.00%
Run 5                           43.33%       